In [81]:
import os

In [82]:
# print the current working directory
print(os.getcwd())
EXTENSION = "mkv"

s:\Shows\Shows-scripts\script_files


In [83]:
# get the list of episodes
episodes_file = os.path.join(
    os.getcwd(),
    "../shows_info_files/TMKOC",
    "episode_name_list.txt",
)

In [84]:
# read episodes file.
# make a hashmap with type <string, int> where key is episode name and value is episode number, read each line from file
# and store it in hashmap

episodes = {}
with open(episodes_file, "r") as f:
    for line in f:
        episode_number, episode_name = line.strip().split(".", 1)
        episode_name = episode_name.strip()
        episode_name = episode_name.strip("?")
        episode_name = episode_name.replace("Taarak Mehta Ka Ooltah Chashmah - ", "")
        episodes[episode_name] = int(episode_number)

In [85]:
# print the first 100 episodes
for episode_name, episode_number in list(episodes.items())[:10]:
    print(f"{episode_number}: {episode_name}")

100: Jethalal And His Family Reach The Theatre
99: A Burglar Snatches Jethalal's Mobile
98: Jethalal Is Excited To Go For Exercise With Babita
97: Jethalal Starts Dieting
96: Babita Suggests Jethalal To Exercise
95: Winner Of Gokuldham's Cultural Dance Program
94: Special Guest In Gokuldham's Cultural Dance Program
93: Taarak Declares A Cultural Dance Program
92: Clash Of Plans
91: Dr. Hathi Gets Stuck In Auto Rickshaw


In [86]:
all_episodes_directory = [
    r"D:\Shows\Taarak Mehta Ka Ooltah Chashmah",
]


# get the list of files in the directory


all_files = []
for directory in all_episodes_directory:
    all_files.extend(os.listdir(directory))

In [87]:
episodes["Bhide And Madhavi's Dream"]

582

In [88]:
# iterate through all files in the directory, and match with the hashmap. Then rename the file with episode number, and name from hashmap
unknown_files = []
unaccounted_for_episodes = episodes.copy()
for file in all_files:
    old_file_name = file
    if file.startswith("S01E"):
        file_name_key = file.split(".", 1)[1]
        file_name_key = file_name_key.replace(f".{EXTENSION}", "")
        file_name_key = file_name_key.strip()
        try:
            unaccounted_for_episodes.pop(file_name_key)
        except Exception as e:
            print(
                f"Error removing {old_file_name}, {file_name_key} from unaccounted_for_episodes"
            )
            print(e)
        continue
    # format file
    file = file.strip()
    # replace name
    file = file.replace("Taarak Mehta Ka Ooltah Chashmah  ", "")
    # remove {EXTENSION}
    file = file.replace(f".{EXTENSION}", "")
    file = file.strip()

    # now since we have a hash map, we can get the episode number from the hashmap
    episode_number = episodes.get(file)
    # if episode number is not found, then add the file to a list
    if episode_number is None:
        print(f"Episode number not found for {file}")
        unknown_files.append(old_file_name)
        continue

    # at this point we got a match.
    # remove the episode from the hashmap
    unaccounted_for_episodes.pop(file)

    # format the episode number to 4 digits
    episode_number = str(episode_number).zfill(4)
    # rename the file
    new_name = f"S01E{episode_number}. {file}.{EXTENSION}"
    print(f"Renaming {file} to {new_name}")
    try:
        pass
        # os.rename(
        #     os.path.join(all_episodes_directory[0], old_file_name),
        #     os.path.join(all_episodes_directory[0], new_name),
        # )
    except Exception as e:
        print(f"Error renaming {file} to {new_name}")
        print(e)

Error removing S01E0001 Taarak Mehta Introduces Himself And Society Members.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0002 Jethalals Son Tapu Decides To Spend His Holiday.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0003 Tapu Has Been The Worry Of Jethalal.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0004 Daya Approaches Taarak Mehta For Help.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0005 Jethalal In Huge Trouble.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0006 Champaklal Gada Goes Missing.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0007 Champaklal Gada Gets A Grand Welcome In Gokuldham Society.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0008 Mumbai Is Hit By Heavy Rains.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0009 Jethalal Is Thrown In An Unlikely Situation.mp4, mp4 from unaccounted_for_episodes
'mp4'
Error removing S01E0

In [89]:
print(len(unknown_files))
print(len(unaccounted_for_episodes))

0
1983


In [90]:
# now for the rest of the files, we know for a fact that the names have got to do atleast something with the names in our list.
# so lets try to find a percentage match between our names and iterate through all our unaccounted for episodes

# for each episode name in unknown files, iterate through all the episode names in the hashmap and find the best match
# if the match is above a certain threshold, then rename the file

# threshold for match

threshold = 0.9
from fuzzywuzzy import fuzz

for unknown_file in unknown_files:
    max_match = 0
    best_match = None
    for episode_name in unaccounted_for_episodes.keys():
        match = fuzz.ratio(unknown_file, episode_name)
        if match > max_match:
            max_match = match
            best_match = episode_name
    if max_match > threshold:
        episode_number = unaccounted_for_episodes.pop(best_match)
        episode_number = str(episode_number).zfill(4)
        new_name = f"S01E{episode_number}. {best_match}.{EXTENSION}"
        print(f"Renaming {unknown_file} to {new_name}")
        try:
            episode_directory = None
            # since we dont know the directory, lets try to rename it through all directories
            for directory in all_episodes_directory:
                try:
                    # os.rename(
                    #     os.path.join(directory, unknown_file),
                    #     os.path.join(directory, new_name),
                    # )
                    episode_directory = directory
                    print(f"Renamed {unknown_file} to {new_name}")
                    break
                except Exception as e:
                    print(f"Error renaming {unknown_file} to {new_name}")
                    print(e)
        except Exception as e:
            print(f"Error renaming {unknown_file} to {new_name}")
            print(e)

In [91]:
print(
    "These episodes are unaccounted for: meaning either they were not renamed somehow, or they were not downloaded"
)
for episode_name in unaccounted_for_episodes.keys():
    print(episode_name)

These episodes are unaccounted for: meaning either they were not renamed somehow, or they were not downloaded
Jethalal And His Family Reach The Theatre
A Burglar Snatches Jethalal's Mobile
Jethalal Is Excited To Go For Exercise With Babita
Jethalal Starts Dieting
Babita Suggests Jethalal To Exercise
Winner Of Gokuldham's Cultural Dance Program
Special Guest In Gokuldham's Cultural Dance Program
Taarak Declares A Cultural Dance Program
Clash Of Plans
Dr. Hathi Gets Stuck In Auto Rickshaw
Dr. Hathi Has Lost 30 Kilos Of Weight
Kite Flying Competition Between Jetha And Sodhi
Gokuldham Society Is In Mood For Celebrations
Celebrations Of Lohri In Gokuldham Society
Sodhi's Efforts
Drunk Roshan Singh Sodhi
Sodhi Has Lied
Lohri Is Round The Corner
Taarak Is Against Child Marriage
Jethalal Has Smelled Trouble
Jethalal Watches Tapu's Marriage
Daya Has Entered A Quarrel
Jethalal Tensed In Tapu's Marriage
Tapu's Marriage Rituals
Gada Convinces Everybody For Tapu's Marriage
Daya Is Excited For Tapu'

In [92]:
# rechecking the files
all_files = []
for directory in all_episodes_directory:
    all_files.extend(os.listdir(directory))

In [93]:
len(all_files)

1997

In [94]:
expected_len = 2000
if len(all_files) == expected_len:
    print(f"All files renamed successfully. Total files: {len(all_files)}")
else:
    print(
        f"Total files: {len(all_files)}. Expected {expected_len}. Please check the files. PS. make sure to check the episode numbering with the created date. Sort by episode number, if something is wrong, you will find an anomaly in created date"
    )

Total files: 1997. Expected 2000. Please check the files. PS. make sure to check the episode numbering with the created date. Sort by episode number, if something is wrong, you will find an anomaly in created date


In [95]:
# lets again find unaccounted episodes but without the hashmap ## Rigorous check
issue_episodes = {}
with open(episodes_file, "r") as f:
    for line in f:
        # find real episode name and number
        episode_number, episode_name = line.strip().split(".", 1)
        episode_name = episode_name.strip()
        episode_name = episode_name.strip("?")
        episode_name = episode_name.replace("Taarak Mehta Ka Ooltah Chashmah - ", "")

        # reconstruct the name it is supposed to have in all files
        episode_number = int(episode_number)
        episode_number = str(episode_number).zfill(4)
        episode_name = f"S01E{episode_number}. {episode_name}.{EXTENSION}"

        # check if the episode is present in all files
        if episode_name not in all_files:
            print("Episode not found: ", episode_name)
            print("episode in question: ", episode_name)
            print("its number: ", episode_number)
            issue_episodes[int(episode_number)] = episode_name

Episode not found:  S01E0100. Jethalal And His Family Reach The Theatre.mkv
episode in question:  S01E0100. Jethalal And His Family Reach The Theatre.mkv
its number:  0100
Episode not found:  S01E0099. A Burglar Snatches Jethalal's Mobile.mkv
episode in question:  S01E0099. A Burglar Snatches Jethalal's Mobile.mkv
its number:  0099
Episode not found:  S01E0098. Jethalal Is Excited To Go For Exercise With Babita.mkv
episode in question:  S01E0098. Jethalal Is Excited To Go For Exercise With Babita.mkv
its number:  0098
Episode not found:  S01E0097. Jethalal Starts Dieting.mkv
episode in question:  S01E0097. Jethalal Starts Dieting.mkv
its number:  0097
Episode not found:  S01E0096. Babita Suggests Jethalal To Exercise.mkv
episode in question:  S01E0096. Babita Suggests Jethalal To Exercise.mkv
its number:  0096
Episode not found:  S01E0095. Winner Of Gokuldham's Cultural Dance Program.mkv
episode in question:  S01E0095. Winner Of Gokuldham's Cultural Dance Program.mkv
its number:  0095


In [96]:
len(episodes)

1983

In [99]:
# so we simply have to download these episodes again.
file_with_urls = os.path.join(
    "../shows_info_files/TMKOC", "tmkoc episode urls till 2000.txt"
)

In [100]:
urls = {}
counter = 1
# assuming each line is just the url
with open(file_with_urls, "r") as f:
    for line in f:
        urls[counter] = line
        counter += 1

In [101]:
len(urls)

0

In [102]:
type(list(issue_episodes.keys())[0])

int

In [103]:
issue_episodes.keys()

dict_keys([100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 300, 299, 298, 297, 296, 295, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 28

In [105]:
# get the links
file_name_with_desc_and_links = os.path.join(
    "../shows_info_files/TMKOC",
    "tmkoc episode with description and everything till 2k.txt",
)

episode_links = {}  # number and link, the number we can get from issue episodes
with open(file_name_with_desc_and_links, "r") as f:
    for i in range(2000):
        link = f.readline()
        thumbnail = f.readline()
        episode_number, episode_name = f.readline().strip().split(".", 1)
        duration = f.readline()
        date = f.readline()
        description = f.readline()
        if int(episode_number) in issue_episodes.keys():
            episode_links[episode_number] = link

In [106]:
episode_links

{'100': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-and-his-family-reach-the-theatre-1000021295?watch=true\n',
 '99': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-a-burglar-snatches-jethalal-s-mobile-1000021294?watch=true\n',
 '98': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-is-excited-to-go-for-exercise-with-babita-1000021293?watch=true\n',
 '97': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-starts-dieting-1000021291?watch=true\n',
 '96': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-babita-suggests-jethalal-to-exercise-1000021288?watch=true\n',
 '95': 'https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-

In [107]:
for i in episode_links.values():
    print(i, end="")

https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-and-his-family-reach-the-theatre-1000021295?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-a-burglar-snatches-jethalal-s-mobile-1000021294?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-is-excited-to-go-for-exercise-with-babita-1000021293?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-jethalal-starts-dieting-1000021291?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-babita-suggests-jethalal-to-exercise-1000021288?watch=true
https://www.sonyliv.com/shows/taarak-mehta-ka-ooltah-chashmah-1700000084/taarak-mehta-ka-ooltah-chashmah-winner-of-gokuldham-s-cultural-dance-program-1000021290?watch=true
ht

In [67]:
# order check
all_files = []
for directory in all_episodes_directory:
    all_files.extend(os.listdir(directory))

In [78]:
test = {}
for i in all_files:
    number = i[4:8]
    name = i[9:]
    test[int(number)] = name

In [79]:
test

{1: 'Taarak Mehta Introduces Himself And Society Members.mp4',
 2: 'Jethalals Son Tapu Decides To Spend His Holiday.mp4',
 3: 'Tapu Has Been The Worry Of Jethalal.mp4',
 4: 'Daya Approaches Taarak Mehta For Help.mp4',
 5: 'Jethalal In Huge Trouble.mp4',
 6: 'Champaklal Gada Goes Missing.mp4',
 7: 'Champaklal Gada Gets A Grand Welcome In Gokuldham Society.mp4',
 8: 'Mumbai Is Hit By Heavy Rains.mp4',
 9: 'Jethalal Is Thrown In An Unlikely Situation.mp4',
 10: 'Champak Chacha Returns Home.mp4',
 11: 'Mobile Phones Could Be Called Best Friends.mp4',
 12: 'Mr Bhide Is Concerned By Sonus Friendship.mp4',
 13: 'Gokuldham Society Celebrates Rakshabandhan.mp4',
 14: 'Jethalal Gets Kidnapped From Work.mp4',
 15: 'Champaklal Informs His Sons Abducters.mp4',
 16: 'The Men Want To Celebrate The Festival Of Janamashtmi.mp4',
 17: 'Champaklal And Tapu Successfully Foil Jethalal And His Friends.mp4',
 18: 'Tapu Lands In Trouble In School.mp4',
 19: 'Jethalal Approaches Chingur Baba For Help.mp4',
 20

In [80]:
len(test)

1983